In [57]:
# fin68_findata.py

from __future__ import annotations
from typing import Optional

import pandas as pd
import vectorbtpro as vbt
from sqlalchemy.engine import Engine

from manage_engine import engine_stockDB  # Engine Postgres bạn đang dùng


def get_fin_symbol(
    symbol: str,
    *,
    engine: Optional[Engine] = None,
    start: Optional[str] = None,
    end: Optional[str] = None,
    **kwargs,
) -> pd.DataFrame:
    """
    Tương tự get_yf_symbol trong docs, nhưng đọc từ Postgres.

    - symbol: mã cổ phiếu, ví dụ 'HPG'
    - start, end: có thể là 'YYYY-MM-DD', 'YYYY-MM-DD UTC', Timestamp,...
      (VBT sẽ hỗ trợ convert về datetime)
    """

    eng = engine or engine_stockDB

    # Chuẩn docs: convert start/end sang datetime cục bộ nếu có
    # (ở đây chủ yếu để bạn giữ cách dùng giống YFData)
    start_str = None
    end_str = None

    if start is not None:
        start_dt = vbt.local_datetime(start)
        start_str = start_dt.strftime("%Y-%m-%d %H:%M:%S")

    if end is not None:
        end_dt = vbt.local_datetime(end)
        end_str = end_dt.strftime("%Y-%m-%d %H:%M:%S")

    safe_symbol = symbol.replace("'", "''")

    sql = f"""
        SELECT
            "DateTime",
            "Open",
            "High",
            "Low",
            "Close",
            "Volume"
        FROM eod.eod_stock_v2
        WHERE "ticker" = '{safe_symbol}'
    """

    if start_str is not None:
        sql += f""" AND "DateTime" >= '{start_str}'"""
    if end_str is not None:
        sql += f""" AND "DateTime" <= '{end_str}'"""

    sql += """ ORDER BY "DateTime" ASC"""

    # Trả về DataFrame giống hệt get_yf_symbol (index = DateTime)
    df = pd.read_sql_query(
        sql,
        eng,
        parse_dates=["DateTime"],
        index_col="DateTime",
    )

    return df


In [58]:
class FINData(vbt.Data):
    """FINData: Data class đọc OHLCV từ Postgres Fin68.

    Dùng đúng pattern YFData trong docs:
    - override fetch_symbol
    - dùng Data.pull để lấy nhiều symbol
    """

    # Engine mặc định, vẫn có thể override bằng tham số engine=...
    default_engine: Engine = engine_stockDB

    @classmethod
    def fetch_symbol(cls, symbol: str, **kwargs):
        """
        Hàm bắt buộc phải override theo docs.

        Data.pull(...) sẽ gọi FINData.fetch_symbol cho từng symbol.
        Ở đây mình forward sang get_fin_symbol(...).
        """

        # Cho phép truyền engine=... trong FINData.pull(...),
        # nếu không có thì dùng default_engine
        engine = kwargs.pop("engine", cls.default_engine)

        return get_fin_symbol(
            symbol,
            engine=engine,
            **kwargs,
        )

In [59]:
# Lấy HPG, VNM từ 2023-01-01 đến nay
fin_data = FINData.pull(
    ["HPG"],
    start="2018-01-01",
    end=None,
    # engine=engine_stockDB  # có thể truyền engine khác nếu muốn
)

In [60]:
fin_data.data['HPG'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1969 entries, 2018-01-02 00:00:00+00:00 to 2025-11-21 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1969 non-null   float64
 1   High    1969 non-null   float64
 2   Low     1969 non-null   float64
 3   Close   1969 non-null   float64
 4   Volume  1969 non-null   float64
dtypes: float64(5)
memory usage: 92.3 KB


In [61]:
open_price = fin_data.get('Open')
close_price = fin_data.get('Close') 

In [62]:
rsi = vbt.RSI.run(open_price)
entries = rsi.rsi.vbt.crossed_below(20)  
exits = rsi.rsi.vbt.crossed_above(80)  


In [63]:
def plot_rsi(rsi, entries, exits):
    fig = rsi.plot()  
    entries.vbt.signals.plot_as_entries(rsi.rsi, fig=fig)  
    exits.vbt.signals.plot_as_exits(rsi.rsi, fig=fig)  
    return fig

In [64]:
plot_rsi(rsi, entries, exits).show()

In [65]:
clean_entries, clean_exits =entries.vbt.signals.clean(exits)

In [66]:
plot_rsi(rsi, clean_entries, clean_exits).show()

In [67]:
clean_entries.vbt.signals.total()

symbol
HPG    3
Name: total, dtype: int32

In [68]:
clean_exits.vbt.signals.total() 

symbol
HPG    3
Name: total, dtype: int32

In [69]:
ranges = clean_entries.vbt.signals.between_ranges(target=clean_exits)

In [70]:
ranges.duration.mean(wrap_kwargs=dict(to_timedelta=True)) 

symbol
HPG   197 days
Name: mean, dtype: timedelta64[ns]

In [71]:
pf=vbt.Portfolio.from_signals(
    close=close_price,
    entries=clean_entries,
    exits=clean_exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [72]:
pf.stats()

Start Index                   2018-01-02 00:00:00+00:00
End Index                     2025-11-21 00:00:00+00:00
Total Duration                       1969 days 00:00:00
Start Value                                       100.0
Min Value                                     70.314465
Max Value                                     208.96419
End Value                                     208.96419
Total Return [%]                              108.96419
Benchmark Return [%]                         203.097345
Position Coverage [%]                         30.015236
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              40.405117
Max Drawdown Duration                 305 days 00:00:00
Total Orders                                          6
Total Fees Paid                                     0.0
Total Trades                                          3
Win Rate [%]                                      100.0
Best Trade [%]                                51

In [73]:
pf.plot(settings=dict(bm_returns=False)).show()

In [74]:
def test_rsi(window=14, wtype="wilder", lower_th=30, upper_th=70):
    rsi = vbt.RSI.run(open_price, window=window, wtype=wtype)
    entries = rsi.rsi_crossed_below(lower_th)
    exits = rsi.rsi_crossed_above(upper_th)
    pf = vbt.Portfolio.from_signals(
        close=close_price, 
        entries=entries, 
        exits=exits,
        size=100,
        size_type='value',
        init_cash='auto')
    return pf.stats([
        'total_return', 
        'total_trades', 
        'win_rate', 
        'expectancy'
    ])
test_rsi()

Total Return [%]    -8.249427
Total Trades                6
Win Rate [%]        66.666667
Expectancy          -1.691066
Name: agg_stats, dtype: object

In [75]:
test_rsi(lower_th=20, upper_th=70)

Total Return [%]    75.008948
Total Trades                3
Win Rate [%]            100.0
Expectancy          25.002983
Name: agg_stats, dtype: object

In [91]:
from itertools import product
lower_ths = range(20, 31)  
upper_ths = range(70, 90)  
th_combs = list(product(lower_ths, upper_ths))  

comb_stats = [
    test_rsi(lower_th=lower_th, upper_th=upper_th)
    for lower_th, upper_th in th_combs
]

In [92]:
comb_stats_df = pd.DataFrame(comb_stats)
comb_stats_df

,Total Return [%],Total Trades,Win Rate [%],Expectancy
agg_stats,75.008948,3,100.0,25.002983
agg_stats,80.341196,3,100.0,26.780399
agg_stats,79.334907,3,100.0,26.444969
agg_stats,87.677181,3,100.0,29.225727
agg_stats,94.156054,3,100.0,31.385351
...,...,...,...,...
agg_stats,177.478448,1,100.0,177.478448
agg_stats,177.478448,1,100.0,177.478448
agg_stats,195.258621,1,NaN,NaN
agg_stats,195.258621,1,NaN,NaN


In [100]:
comb_stats_df.index = pd.MultiIndex.from_tuples(
    th_combs, 
    names=['lower_th', 'upper_th'])
comb_stats_df.sort_values('Total Return [%]',ascending=False).dropna()

Total Return [%]  Total Trades  Win Rate [%]  Expectancy
lower_th upper_th                                                          
25       86              226.776650             1    100.000000  226.776650
26       86              226.776650             1    100.000000  226.776650
         85              226.776650             1    100.000000  226.776650
25       85              226.776650             1    100.000000  226.776650
24       86              223.899371             1    100.000000  223.899371
...                             ...           ...           ...         ...
30       76              -18.898770             3     33.333333   -8.465311
         74              -18.898770             3     33.333333   -8.465311
         72              -19.021487             4     50.000000   -6.102525
29       73              -19.880021             3     33.333333   -8.904842
30       73              -22.667110             3     33.333333  -10.153260

[187 rows x 4 columns]

In [99]:
comb_stats_df['Expectancy'].dropna().vbt.heatmap().show()

In [115]:
windows = list(range(8, 21))
wtypes = ["simple", "exp", "wilder"]
lower_ths = list(range(20, 31))
upper_ths = list(range(70, 81))

In [116]:
rsi = vbt.RSI.run(
    open_price, 
    window=windows, 
    wtype=wtypes, 
    param_product=True)
rsi.rsi.columns

MultiIndex([( 8, 'simple', 'HPG'),
            ( 8,    'exp', 'HPG'),
            ( 8, 'wilder', 'HPG'),
            ( 9, 'simple', 'HPG'),
            ( 9,    'exp', 'HPG'),
            ( 9, 'wilder', 'HPG'),
            (10, 'simple', 'HPG'),
            (10,    'exp', 'HPG'),
            (10, 'wilder', 'HPG'),
            (11, 'simple', 'HPG'),
            (11,    'exp', 'HPG'),
            (11, 'wilder', 'HPG'),
            (12, 'simple', 'HPG'),
            (12,    'exp', 'HPG'),
            (12, 'wilder', 'HPG'),
            (13, 'simple', 'HPG'),
            (13,    'exp', 'HPG'),
            (13, 'wilder', 'HPG'),
            (14, 'simple', 'HPG'),
            (14,    'exp', 'HPG'),
            (14, 'wilder', 'HPG'),
            (15, 'simple', 'HPG'),
            (15,    'exp', 'HPG'),
            (15, 'wilder', 'HPG'),
            (16, 'simple', 'HPG'),
            (16,    'exp', 'HPG'),
            (16, 'wilder', 'HPG'),
            (17, 'simple', 'HPG'),
            (17,    

In [117]:
lower_ths_prod, upper_ths_prod = zip(*product(lower_ths, upper_ths))
len(lower_ths_prod)  

len(upper_ths_prod)


lower_th_index = vbt.Param(lower_ths_prod, name='lower_th')  
entries = rsi.rsi_crossed_below(lower_th_index)
entries.columns


upper_th_index = vbt.Param(upper_ths_prod, name='upper_th')
exits = rsi.rsi_crossed_above(upper_th_index)
exits.columns

MultiIndex([(70,  8, 'simple', 'HPG'),
            (70,  8,    'exp', 'HPG'),
            (70,  8, 'wilder', 'HPG'),
            (70,  9, 'simple', 'HPG'),
            (70,  9,    'exp', 'HPG'),
            (70,  9, 'wilder', 'HPG'),
            (70, 10, 'simple', 'HPG'),
            (70, 10,    'exp', 'HPG'),
            (70, 10, 'wilder', 'HPG'),
            (70, 11, 'simple', 'HPG'),
            ...
            (80, 17, 'wilder', 'HPG'),
            (80, 18, 'simple', 'HPG'),
            (80, 18,    'exp', 'HPG'),
            (80, 18, 'wilder', 'HPG'),
            (80, 19, 'simple', 'HPG'),
            (80, 19,    'exp', 'HPG'),
            (80, 19, 'wilder', 'HPG'),
            (80, 20, 'simple', 'HPG'),
            (80, 20,    'exp', 'HPG'),
            (80, 20, 'wilder', 'HPG')],
           names=['upper_th', 'rsi_window', 'rsi_wtype', 'symbol'], length=4719)

In [128]:
pf = vbt.Portfolio.from_signals(
    close=close_price, 
    entries=entries, 
    exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

In [130]:
stats_df = pf.stats([
    'end_value',
    'total_return', 
    'total_trades', 
    'win_rate', 
    'expectancy'
], agg_func=None)  
stats_df.sort_values("Total Return [%]",ascending=False)

,,,,,End Value,Total Return [%],Total Trades,Win Rate [%],Expectancy
lower_th,upper_th,rsi_window,rsi_wtype,symbol,,,,,
26,80,20,wilder,HPG,482.607256,382.607256,2,100.000000,191.303628
25,80,20,wilder,HPG,482.607256,382.607256,2,100.000000,191.303628
27,80,20,wilder,HPG,447.506399,347.506399,2,100.000000,173.753200
29,80,20,wilder,HPG,396.724046,296.724046,2,100.000000,148.362023
28,80,20,wilder,HPG,393.264607,293.264607,2,100.000000,146.632304
...,...,...,...,...,...,...,...,...,...
25,70,9,simple,HPG,110.924349,-39.569475,26,53.846154,-2.793557
24,76,15,simple,HPG,94.926341,-40.208389,8,42.857143,-8.238750
23,71,9,simple,HPG,110.085497,-40.657432,22,45.454545,-3.428317


In [129]:
pf.stats()

d:\stockvip\vectorbtLearn\venv\Lib\site-packages\vectorbtpro\generic\stats_builder.py:789: VBTWarning: Object has multiple columns. Aggregated some metrics using <function mean at 0x0000022D159619E0>. Pass either agg_func=None or per_column=True to return statistics per column. Pass column to select a single column or group.


Start Index                      2018-01-02 00:00:00+00:00
End Index                        2025-11-21 00:00:00+00:00
Total Duration                          1969 days 00:00:00
Start Value                                     121.945912
Min Value                                        74.786363
Max Value                                       160.567066
End Value                                       135.802026
Total Return [%]                                 15.927428
Benchmark Return [%]                            203.097345
Position Coverage [%]                            37.932389
Max Gross Exposure [%]                           98.601399
Max Drawdown [%]                                 48.093316
Max Drawdown Duration         1073 days 19:09:05.454545456
Total Orders                                     22.074168
Total Fees Paid                                        0.0
Total Trades                                     11.288832
Win Rate [%]                                     65.1776

In [123]:
pf.getsize()

d:\stockvip\vectorbtLearn\venv\Lib\site-packages\vectorbtpro\utils\module_.py:354: VBTWarning: Please install dill. See https://pypi.org/project/dill/.


'9.3 MB'